In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import os

class SkinConditionDataset(tfds.core.GeneratorBasedBuilder):
    """DatasetBuilder for skin condition detection."""

    VERSION = tfds.core.Version('1.0.0')

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=("Dataset untuk deteksi kondisi kulit seperti kulit kering, berminyak, keriput, berjerawat, dan merah."),
            features=tfds.features.FeaturesDict({
                'image': tfds.features.Image(shape=(224, 224, 3)),
                'label': tfds.features.Tensor(shape=(5,), dtype=tf.float32),
            }),
            supervised_keys=('image', 'label'),
        )

    def _split_generators(self, dl_manager):
        # Mengambil data dari path lokal
        path = dl_manager.manual_dir
        return {
            'train': self._generate_examples(path / 'train'),
            'val': self._generate_examples(path / 'val'),
            'test': self._generate_examples(path / 'test'),
        }

    def _generate_examples(self, path):
        """Yields examples."""
        labels_df = pd.read_csv(os.path.join(path, 'labels.csv'))
        for _, row in labels_df.iterrows():
            yield row['filename'], {
                'image': os.path.join(path, 'images', row['filename']),
                'label': row[['kulit_kering', 'kulit_berminyak', 'kulit_keriput', 'kulit_berjerawat', 'kulit_merah']].to_numpy()
            }

# Path ke dataset lokal
dataset_path = '/path_to_dataset'
builder = SkinConditionDataset(data_dir=dataset_path)
builder.download_and_prepare()
ds_train = builder.as_dataset(split='train')
ds_val = builder.as_dataset(split='val')
ds_test = builder.as_dataset(split='test')


In [ ]:
# Fungsi untuk augmentasi data
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

ds_train = ds_train.map(augment).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Pelatihan model
model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10
)

# Evaluasi model pada data testing
loss, accuracy = model.evaluate(ds_test)
print(f'Testing Loss: {loss}, Testing Accuracy: {accuracy}')
